## Vlasov equations

We want to solve Vlasov equations on 1  (in $S^1$).

The equations are: 

\begin{align}
\partial_t f + p \partial_x f &= -q E \partial_p f \\
\partial_t E = - J \\
\partial_x E = \rho \\
\end{align}

Where $f = f(t,x,p)$ is a distribution function in phase espace. 
Note that the invarian volume element is, 

$$
dP = dp_x/p_0
$$ 

We define, the particle number with respect to an observer with four-velocity $u^a$,

$$
N = \int f(x,p)(-u \cdot p)\; dP
$$ 

Thus, for the observer at rest in the coordinate system $(t,x)$ we get

$$
N = \int f(x,p) p_0 \; dP = \int f(x,p) \; dp_x
$$ 

Otherwise one has the four-vector particle density,

$$
N^a = \int f(x,p) p^a \; dP = \int f(x,p) \frac{p^a}{p_0} \; dp_x
$$

Thus, 

$$
N^x = \int f(x,p) \frac{p_x}{p_0} \; dp_x
$$

Likewise we have the energy-momentum tensor,

$$
T^{ab} = \int p^a p^b f(x,p) dP 
$$

So, 

$$
T^{00} = \int p^0 f(x,p) dp_x = m \int \sqrt{1 + p^2/m^2} dp
$$




\begin{align}
\rho(t,x) &:= q\int f(t,x,p) \; dp - n_0 \\
n_0 &:= q \int \int f(t,x,p) \; dp \; dx / V\\
J(t,x) &:= q \int v f(t,x,p) \; dp, \\
v &:= \frac{\frac{p}{m}}{\sqrt{1 + \frac{p^2}{m^2}}} \\ 
\end{align}

And $E = E(t,x)$ is the electric field.


The equilibrium distribution function is: 

$$
f(\gamma) = \frac{\gamma^2 \beta}{\theta K_2(\frac{1}{\theta})}\; e^{-\frac{\gamma}{\theta}} \;\;\;\; \gamma = \frac{1}{\sqrt{1 - \beta^2}}
$$ 

or

$$
f(p) = \frac{1}{4\pi m^3 c^3 \theta K_2(\frac{1}{\theta})} \; e^{-\frac{\gamma(p)}{\theta}} \;\;\;\;\; \gamma(p) = \sqrt{1 + (\frac{p}{m})^2}
$$

and $K_2$ is the Bessel function of second kind.

Actually in de code we normalize things differently and define:,
    
$$
    f(p) = f_0 e^{\frac{1-\gamma(p)}{\theta}} \;\;\;\;\; \gamma(p) = \sqrt{1 + (\frac{p}{m})^2},
$$

In [1]:
using Plots
using Statistics
using FFTW
FFTW.set_provider!("mkl")
#import Pkg; Pkg.add("FileIO")
using FileIO
using JLD2
using Base.Threads
using Distributions
#Pkg; Pkg.add("DistributedArrays")
println("nthreads = $(nthreads())")
using Printf

nthreads = 2


In [2]:
plots = false #sacamos los plots
outputs = false #sacamos los outputs procesados

false

In [38]:
include("aux_functions_vlasov.jl")

In [40]:
run_name = "l_"

D_order = 4 # order of finite difference OPERATORS

landau = false
two_streams = false
landau = true
#two_streams = true
#const Np = 201 # we take even since we need positive and negative values
#const Lx = 1
#const Lp = 0.5 # para cada lado

if landau
    if true #undamped
        const Lx = 39.738 
        const Nx = 3522
        const n = 2
        run_name = run_name * "landau_undamped"
    else #damped 
        const Lx = 7.455
        const Nx = 930
        const n = 15
        run_name = run_name * "landau_damped"
    end

    const Np = 400
    const Lp = 1.0
    
    α = 0.001
    k = 2*π*n/Lx
    θ = 0.001
    run_name = run_name * "$(Nx)_$(Np)_alp3_n$(n)_Th3"
elseif two_streams
    const Nx = 500 # usar par para Fourier transform
    const Lx = 10.0
    const Np = 200
    const Lp = 1.0
    run_name = run_name * "two_streams_"
    const n = 1
    k = 2*π*n/Lx
    α = 0.1
    θ = 0.005
    vel = 0.2
    run_name = run_name * "$(Nx)_$(Np)_v_02_n$(n)_o4_Th005"
end

dx = Lx/Nx # periodic boundary conditions
dp = 2*Lp/(Np-1) # Dichlet... or whatever but not periodic

const m = 1
const e =-1
#const q = 1

const κ = 2π/Lx

println("omega = $(1+3*θ*k^2/2)")

grid = Grid(Nx, Np, Lx, Lp, dx, dp, true, false, 4)

par_grid = (Nx, dx, Np, dp)

p = [get_p(j,dp,Np)/m for j ∈ 1:Np]
v = [p[j]/sqrt(1+p[j]^2) for j ∈ 1:Np];

run_name


omega = 1.0001500026654444
omega = 1.0001500026654444


"l_landau_undamped3522_400_alp3_n2_Th3"

In [5]:
#E = zeros(Nx)
#ϕ = zeros(Nx)
ρ = zeros(Nx) #charge density
S = zeros(Nx) #carge current
E_K = zeros(Nx) # kinetic energy
#E_E = zeros(Nx) # Electromagnetic energy
P = zeros(Nx) # Momentum
du = zeros(Nx * (Np+1)); # contains f and E


## Initial data

In [6]:
u = zeros(Nx * (Np+1))

pars = (Nx, dx, Lx, Np, dp, Lp, κ, e)



if landau
    pars_f = (m, θ, α, k)
    u = generate_initial_data!(landau_rel_dist, u, pars_f, pars);
elseif two_streams
    pars_f = (m, θ, α, k, vel)
    u = generate_initial_data!(counter_streams_rel_dist, u, pars_f, pars);
end

if plots

    F = reshape(u[1:Nx*Np],(Nx,Np));

    #plot(F[Nx÷2,:])
    plot_matrix(F)
    png("initial_dist_" * run_name)
end

n0 = -0.1442253930038297
n0 = -0.9999999999999999


In [7]:
if plots
    heatmap(F,
        #c = cgrad([:blue,:white,:red]),
        fc = :ocean,
        linealpha = 0.8, fillalpha=0.8,
        yflip = true,
        xlabel = "p", ylabel = "x",
        title = "Distribution function"
    )

    png("initial_conf_heat_" * run_name)
end

In [8]:
if plots 
    E_i = deepcopy(u[Nx*Np+1:end])
    get_current!(u, S, (Nx, dx, Np, dp, v, m, e));
    get_density!(u, ρ, (Nx, dx, Np, dp, m, e))
    plot(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))

    x = [(i-1)*dx for i in 1:Nx]

    plot(layout=(2,2))
    plot!(subplot=1,x,ρ, title = "density", legend = :false)
    plot!(subplot=2,x,E_i, title = "Electric Field", legend = :false)
    plot!(subplot=3,x,S, title = "Current", legend = :false)
    plot!(subplot=4,x,E_K, title = "Kinetic Energy", legend = :false)

    png("initial_conf_" * run_name)
end

## Time evolution

In [9]:
t = 0.0
t_i = 0.0
if two_streams
    t_f = 200.0
    M = 20001
    M_g = 500 + 1 #number of outputs, starting from the initial data
end
if landau
    t_f = 200.0
    M = 20001
    M_g = 500 + 1 
end
    
dt = t_f / (M-1)
par_evolv = (t_i, t_f, M, M_g, dt)



(0.0, 200.0, 20001, 501, 0.01)

In [10]:
dvx = zeros(Nx)
dvp = zeros(Np)
k1 = zeros(Nx*(Np+1))
k2 = zeros(Nx*(Np+1))
k3 = zeros(Nx*(Np+1))
k4 = zeros(Nx*(Np+1))
p_F = (dx, dp, Nx, Np, v, S, dvx, dvp, D_order)
par_RK = (k1, k2, k3, k4)

if outputs
    # total quantities 
    Energy_K = zeros(M_g)
    Energy_E = zeros(M_g)
    EField_T = zeros(M_g)
    p_T = zeros(M_g)
    n_T = zeros(M_g)
    S_T = zeros(M_g)
    E_E = 0.0
    #T = zeros(M_g)
end

In [11]:
j = 1

if outputs
    Energy_K[j]  = sum(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))*dx
    Energy_E[j]  = get_E_energy(u,(Nx, dx, Np, dp))
    EField_T[j] = sum(u[Nx*Np+1:end])*dx
    p_T[j] = sum(get_momentum!(u,P,(Nx, dx, Np, dp)))*dx

    get_density!(u, ρ, (Nx, dx, Np, dp, m, e))
    get_current!(u, S, (Nx, dx, Np, dp, v, m, e))
    n_T[j] = get_total_density!(ρ,(Nx, dx))
    S_T[j] = sum(S)/n_T[j]/Nx
    #T[1] = var(u[N+1:2N])
end

In [31]:
run_pars = Dict("run_name" => run_name, "par_grid" => par_grid, "par_evolv" => par_evolv, "p_Ini" => pars_f)
if outputs
    run_data = Dict()
    run["Energy_E_$j"] = Energy_E[j]
    run["Energy_K_$j"] = Energy_K[j]
    run["E_f$j"] = u[Nx*Np+1:end]
    run["n_F$j"] = ρ
    run["S_F$j"] = S
    run["E_T$j"] = EField_T[j] 
    run["S_T$j"] = S_T[j]
    run["n_T$j"] = n_T[j]
end


file_name = "Results/"* run_name * ".jld2"
#rm(file_name)
j = 1
tiempo = @sprintf("%05d", j)
field_name = "u"

save(file_name, run_pars)

if false
file = jldopen(file_name, "r+")
close(file)
end

jldopen(file_name, "a+") do file
    file[field_name * "/u_$(tiempo)"] = u;
end


401000-element Vector{Float64}:
 5.204043953062454e-22
 5.114951894697227e-22
 5.027373633077463e-22
 4.939126264611971e-22
 4.852783865043863e-22
 4.7655046189880445e-22
 4.68001953996986e-22
 4.593929130663621e-22
 4.509581644579783e-22
 4.424334433877347e-22
 ⋮
 0.0008908779200140665
 0.0007919798860518507
 0.000693050655799926
 0.0005940941261242148
 0.0004951141949575463
 0.0003961147611496932
 0.00029709972431358914
 0.00019807298467207035
 9.903844290513836e-5

### Temporal evolution: This takes a while!

In [32]:
#include("aux_functions_vlasov.jl")

for k in 2:M
    RK4_Step!(F!,u,t,dt,p_F,par_RK)
    global t = t + dt
    if (k-1) % (M÷(M_g-1)) == 0
        local j = (k-1)÷(M÷(M_g-1))+1
        if outputs
            Energy_K[j]  = sum(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))*dx
            Energy_E[j]  = get_E_energy(u,(Nx, dx, Np, dp))
            EField_T[j] = sum(u[Nx*Np+1:end])*dx
            p_T[j] = sum(get_momentum!(u,P,(Nx, dx, Np, dp)))*dx
            get_density!(u, ρ, (Nx, dx, Np, dp, m, e))
            get_current!(u, S, (Nx, dx, Np, dp, v, m, e))
            n_T[j] = get_total_density!(ρ,(Nx, dx))
            S_T[j] = sum(S)/n_T[j]/Nx
        end
        
        local tiempo = @sprintf("%05d", j)
        jldopen(file_name, "a+") do file
            file[field_name * "/u_$(tiempo)"] = u
        end
        Energy = sum(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))*dx + get_E_energy(u,(Nx, dx, Np, dp))
        println("j = $j , t = $t, k = $k, nthreads = $(nthreads()), Energy = $Energy")
        
    end
end

j = 2 , t = 0.8000000000000005, k = 41, nthreads = 2, Energy = 1.0240191719737541
j = 3 , t = 1.2000000000000008, k = 81, nthreads = 2, Energy = 1.0240191719737313
j = 4 , t = 1.6000000000000012, k = 121, nthreads = 2, Energy = 1.0240191719737017
j = 5 , t = 2.0000000000000013, k = 161, nthreads = 2, Energy = 1.0240191719736678
j = 6 , t = 2.399999999999993, k = 201, nthreads = 2, Energy = 1.0240191719736313
j = 7 , t = 2.7999999999999843, k = 241, nthreads = 2, Energy = 1.0240191719735956
j = 8 , t = 3.1999999999999758, k = 281, nthreads = 2, Energy = 1.0240191719735627
j = 9 , t = 3.5999999999999672, k = 321, nthreads = 2, Energy = 1.0240191719735345
j = 10 , t = 3.9999999999999587, k = 361, nthreads = 2, Energy = 1.0240191719735134
j = 11 , t = 4.399999999999951, k = 401, nthreads = 2, Energy = 1.0240191719734997
j = 12 , t = 4.799999999999942, k = 441, nthreads = 2, Energy = 1.0240191719734937
j = 13 , t = 5.199999999999934, k = 481, nthreads = 2, Energy = 1.0240191719734955
j = 14

j = 124 , t = 49.5999999999987, k = 4921, nthreads = 2, Energy = 1.0240191719433753
j = 125 , t = 49.99999999999862, k = 4961, nthreads = 2, Energy = 1.0240191719085001
j = 126 , t = 50.39999999999854, k = 5001, nthreads = 2, Energy = 1.024019171847666
j = 127 , t = 50.79999999999846, k = 5041, nthreads = 2, Energy = 1.0240191717531062
j = 128 , t = 51.19999999999838, k = 5081, nthreads = 2, Energy = 1.0240191716237894
j = 129 , t = 51.5999999999983, k = 5121, nthreads = 2, Energy = 1.0240191714705231
j = 130 , t = 51.999999999998224, k = 5161, nthreads = 2, Energy = 1.024019171314935
j = 131 , t = 52.399999999998144, k = 5201, nthreads = 2, Energy = 1.024019171177894
j = 132 , t = 52.799999999998064, k = 5241, nthreads = 2, Energy = 1.024019171064771
j = 133 , t = 53.199999999997985, k = 5281, nthreads = 2, Energy = 1.024019170969583
j = 134 , t = 53.599999999997905, k = 5321, nthreads = 2, Energy = 1.0240191709025495
j = 135 , t = 53.999999999997826, k = 5361, nthreads = 2, Energy = 

j = 245 , t = 98.00000000001323, k = 9761, nthreads = 2, Energy = 1.0240166367955408
j = 246 , t = 98.40000000001343, k = 9801, nthreads = 2, Energy = 1.0240166815069387
j = 247 , t = 98.80000000001364, k = 9841, nthreads = 2, Energy = 1.024016489165045
j = 248 , t = 99.20000000001384, k = 9881, nthreads = 2, Energy = 1.0240161265946426
j = 249 , t = 99.60000000001405, k = 9921, nthreads = 2, Energy = 1.0240158103790173
j = 250 , t = 100.00000000001425, k = 9961, nthreads = 2, Energy = 1.0240155181482309
j = 251 , t = 100.40000000001446, k = 10001, nthreads = 2, Energy = 1.0240152444581392
j = 252 , t = 100.80000000001466, k = 10041, nthreads = 2, Energy = 1.0240149958187275
j = 253 , t = 101.20000000001487, k = 10081, nthreads = 2, Energy = 1.0240147651034543
j = 254 , t = 101.60000000001507, k = 10121, nthreads = 2, Energy = 1.0240145553621087
j = 255 , t = 102.00000000001528, k = 10161, nthreads = 2, Energy = 1.02401436212573
j = 256 , t = 102.40000000001548, k = 10201, nthreads = 2

j = 364 , t = 145.60000000001256, k = 14521, nthreads = 2, Energy = 1.0240108555746605
j = 365 , t = 146.0000000000122, k = 14561, nthreads = 2, Energy = 1.0240108714573228
j = 366 , t = 146.40000000001183, k = 14601, nthreads = 2, Energy = 1.0240108515653743
j = 367 , t = 146.80000000001147, k = 14641, nthreads = 2, Energy = 1.024010802603954
j = 368 , t = 147.2000000000111, k = 14681, nthreads = 2, Energy = 1.0240108128617877
j = 369 , t = 147.60000000001074, k = 14721, nthreads = 2, Energy = 1.0240108051210841
j = 370 , t = 148.00000000001037, k = 14761, nthreads = 2, Energy = 1.024010796879477
j = 371 , t = 148.40000000001, k = 14801, nthreads = 2, Energy = 1.0240108042963114
j = 372 , t = 148.80000000000965, k = 14841, nthreads = 2, Energy = 1.0240108007411095
j = 373 , t = 149.20000000000928, k = 14881, nthreads = 2, Energy = 1.0240107690351
j = 374 , t = 149.60000000000892, k = 14921, nthreads = 2, Energy = 1.0240107154688631
j = 375 , t = 150.00000000000855, k = 14961, nthreads

j = 483 , t = 193.19999999996926, k = 19281, nthreads = 2, Energy = 1.0240100305438595
j = 484 , t = 193.5999999999689, k = 19321, nthreads = 2, Energy = 1.0240100437441375
j = 485 , t = 193.99999999996854, k = 19361, nthreads = 2, Energy = 1.0240100515393562
j = 486 , t = 194.39999999996817, k = 19401, nthreads = 2, Energy = 1.0240100622431045
j = 487 , t = 194.7999999999678, k = 19441, nthreads = 2, Energy = 1.0240100580696645
j = 488 , t = 195.19999999996745, k = 19481, nthreads = 2, Energy = 1.0240100538362384
j = 489 , t = 195.59999999996708, k = 19521, nthreads = 2, Energy = 1.0240100465668278
j = 490 , t = 195.99999999996672, k = 19561, nthreads = 2, Energy = 1.0240100609564504
j = 491 , t = 196.39999999996635, k = 19601, nthreads = 2, Energy = 1.0240100572325512
j = 492 , t = 196.799999999966, k = 19641, nthreads = 2, Energy = 1.0240100516571864
j = 493 , t = 197.19999999996563, k = 19681, nthreads = 2, Energy = 1.0240100450227587
j = 494 , t = 197.59999999996526, k = 19721, nt

In [14]:
#file = jldopen(file_name, "r+")
#close(file)

In [15]:
if outputs
    run = Dict("run_name" => run_name, "par_grid" => par_grid, "par_evolv" => par_evolv, "p_Ini" => pars_f, "Energy_E" => Energy_E, "Energy_K" => Energy_K, "E_f" => u[Nx*Np+1:end], "n_F" => ρ, "S_F" => S, "E_T"=> EField_T, "S_T" => S_T, "n_T" => n_T)
    save("Results/"* run_name * "th$(nthreads()).jld2", run)
end

### Shows the final data, warning lot of  memory for big grids! 

In [16]:
if plots
    F = reshape(u[1:Nx*Np],(Nx,Np));
    plot_matrix(F, camera=(90,70))
    #plot_matrix(F; fc=:heat)


    png("final_conf_" * run_name)
end

In [17]:
if plots 
    heatmap(F,
        #c = cgrad([:blue,:white,:red]),
        fc = :ocean,
        linealpha = 0.8, fillalpha=0.8,
        yflip = true,
        xlabel = "p", ylabel = "x",
        title = "Distribution function at t = $(t_f)"
    )

    png("final_conf_heat_" * run_name)
end

In [18]:
if plots
    ρ_f = zeros(Nx)
    E_f = zeros(Nx)
    ϕ_f = zeros(Nx)
    S_f = zeros(Nx)


    get_density!(u, ρ_f, (Nx, dx, Np, dp, m, e))
    n0 = get_total_density!(ρ_f, (Nx, dx))
    println("n0 = $(n0)")
    get_ϕ!(ϕ_f, ρ_f .- e*n0, κ)
    get_E_from_ϕ!(ϕ_f,E_f,dx)


    plot(x,E_f,label="from final density", ls=:dash, lw=2)
    plot!(x,E_i,label="E_initial")
    plot!(x,u[Nx*Np+1:end], label="E_final")

    png("Efield_th$(nthreads())_" * run_name)
end

In [19]:
if plots
    factor = 200
    plot(layout=(2,2), size=(800,600))
    plot!(subplot=1, (Energy_K[1:end] .- Energy_K[1]), label="Energy_K")
    plot!(subplot=1, (Energy_E .- Energy_E[1]), label="Energy_E")
    #plot!(subplot=1, Energy_K, label="Energy_K")
    #plot!(subplot=1, Energy_E[1:400], label="Energy_E")
    plot!(subplot=2, (Energy_K + Energy_E) ./ (Energy_K[1] + Energy_E[1]) .- 1, label="Total Energy")
    plot!(subplot=3, n_T .+ 1.0, label="Total density")
    plot!(subplot=4, S_T, label="Total Current")

    png("final_totals_th$(nthreads())_" * run_name)
end

In [20]:
#plot(ρ_f)

In [21]:
#plot(p_T)

In [22]:
#Energy_K = load(run_name * "th$(nthreads())_results.jld2", "Energy_K")
#Energy_E = load(run_name * "th$(nthreads())_results.jld2", "Energy_E")
#plot(Energy_K .- Energy_K[1])
#plot!(Energy_E)


In [23]:
if plots
    println("M = $M, M_g = $M_g")
    tt = 0:dt*(M-1)/(M_g-1):t_f

    if true #undamped
        plot(tt,Energy_K)
        plot!(tt, 1.0005005 .+ 0.0000196*cos.(0.562*tt .- π/2).^2)
        plot!(tt, 1.0005103 .+ 0.0000196/2*cos.(0.562*2*tt .- π))
    else
        plot(tt,log10.(Energy_E))
        plot!(tt, log10.(10^(-7)*1.450*cos.(1.512*tt .- 0).^(2).*exp.(-0.03.* tt)))
    end
end

Los parámetros de fiteo son: 

### Caso undamped

    Lx = 39.738, Nx = 3522, Np = 200, Lp = 0.5

    α = 0.01
    n = 4
    k = 2*π*n/Lx
    θ = 0.001

$E_K = a + b*cos(\omega*t + \alpha)^2$ 

Tenemos $a = 1.0005005$, $b = 0.0000196$ $\omega = 0.562$, $\alpha = -\pi/2$

$E_K = a' + d'*cos(\omega' * t + \alpha')$ 

Tenemos $a'= 1.0005103$, $b'= 0.0000196/2$ $\omega' = 0.562*2$ $\alpha'= - π$

### Caso damped

    Lx = 7.455, Nx = 930, Np = 200, Lp = 0.5

    α = 0.01
    n = 15
    k = 2*π*n/Lx
    θ = 0.001

$E_E = b*cos(\omega*t)^2 e^{-\gamma * t}$ 

Tenemos: $b = 1.450 \; 10^{-7}$, $\omega = 1.512$, $\gamma = 0.03$


#### Note: 

Since we are using a $4\pi$ in the equation for E dot, we need to change E and t to get to the equations in the SHARP paper, for that we have to change time by a factor $\sqrt{4\pi}$. 

Using this factor we get:

#### undamped: #### 

$\omega = \; \sqrt{4π} * 0.562 \;= \;1.9922 $

#### damped: #### 

$\omega = \sqrt{4π} * 1.512 = 5.3599$, $\gamma = \sqrt{4π} * 0.03 = 0.1063$

In [24]:
#sqrt(4π)* 0.03

#5.36/4

In [25]:
#using LsqFit

In [26]:

#p0 = [0 ; 1; 0]
#@. model(x, p) = p[1]*sin(x*p[2])^2 * exp(-x*p[3]) #*cos(x*p[5] + p[6])

In [27]:
#t_series = [dt*(i-1)*(M-1)/(M_g-1) for i in 1:M_g];
#fit = curve_fit(model, t_series, Energy_E, p0);
#fit.param

In [28]:
#20*sqrt(4π )